In [1]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 921.7 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 32.4 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.4 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.2 MB/s eta 0:00:0000:01
  Using cached huggingface_hub-0.15.1-py3-none-any.whl (236 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.4/770.4 kB 10.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 27.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.4 MB/s eta 0:00:00:00:01
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=f08826dead458507e11b6953a6e41a24af5d1561fcf0756c8f22ff91fcc6f74e
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f

In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 5.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 3.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 2.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 3.0 MB/s eta 0:00:00a 0:00:01


# finetune 模型

In [2]:
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
from sentence_transformers import InputExample
from torch.utils.data import DataLoader
from sentence_transformers import losses
import torch

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 从hf加载模型

In [4]:
modelB = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

In [11]:
# dataset_id = "embedding-data/sentence-compression"
# dataset = load_dataset(dataset_id)

In [12]:
import os
filenames = os.listdir('topwar_faq')
filenames

['3_faq.faq', '1_faq.faq', '4_faq.faq', '2_faq.faq']

In [107]:
def parse_faq(file_content,QA_SEP='====='):
    arr = file_content.split(QA_SEP)
    list_arr = []
    for item in arr:
        question, answer = item.strip().split("\n", 1)
        question = question.replace("Question: ", "")
        answer = answer.replace("Answer: ", "")
        list_arr.append((answer,question))
    return list_arr

all_datas = []
for fn in filenames:
    with open(f"topwar_faq/{fn}") as f:
        data = f.read()
        all_datas += parse_faq(data)
print(f"data size:{len(all_datas)}")

data size:126


In [111]:
train_examples = []

for i in range(len(all_datas)):
    example = all_datas[i]
    train_examples.append(InputExample(texts=[example[0], example[1]]))

In [196]:

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=64)
train_loss = losses.MultipleNegativesRankingLoss(model=modelB)
num_epochs = 20
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data

In [3]:
from sentence_transformers import evaluation,util
# sentences1 = ['专属技能碎片可以通过多种途径获得，例如礼包商城-特惠礼包界面可以购买专属技能碎片礼包', '您可以在军火库制作对应装备的界面查看到您拥有的图纸']
# sentences2 = ['专属技能碎片在哪里获得？', '在哪里能看到我刚刚获得的装备图纸?']
# scores = [0.9, 0.9]
# evaluator = evaluation.EmbeddingSimilarityEvaluator(sentences1, sentences2,scores)

In [197]:
torch.cuda.empty_cache()
modelB.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
           output_path='./finetuned-sentence-embedding',
           evaluation_steps=10,
          warmup_steps=warmup_steps)

Epoch: 100%|██████████| 10/10 [00:28<00:00,  2.85s/it]


## 从本地加载finetuned的模型

In [4]:
modelB = SentenceTransformer('./finetuned-sentence-embedding')

In [10]:
input_sentences_a = ['专属技能碎片可以通过多种途径获得，例如礼包商城-特惠礼包界面可以购买专属技能碎片礼包', 
                  '中国首都在北京',
                  '美国首都在华盛顿']
input_sentences_q = ['专属技能碎片在哪里？',
                   '中国首都在哪里？',
                   '美国首都在哪里？']

embeddings = modelB.encode(input_sentences_a+input_sentences_q)

In [11]:
from sentence_transformers import evaluation,util

util.cos_sim(embeddings[:3],embeddings[3:])

tensor([[ 0.7692, -0.0653, -0.1117],
        [-0.0656,  0.8789,  0.4908],
        [-0.0902,  0.5818,  0.8457]])

In [198]:
input_answer  = []
input_question  = []
for i, (a,b) in enumerate (all_datas):
        input_answer.append(a)
        input_question.append(b)

In [199]:
emb_answer = modelB.encode(input_answer)
emb_question = modelB.encode(input_question)

In [200]:
len(input_question),len(input_answer)

(126, 126)

In [201]:
import numpy as np
def similarity(v1,v2):
    dot_product = np.dot(v1, v2)

    magnitude_v1 = np.linalg.norm(v1)
    magnitude_v2 = np.linalg.norm(v2)

    return dot_product / (magnitude_v1 * magnitude_v2)

In [ ]:
for i in range(len(input_question)):
    sims = similarity(emb_answer[i],emb_question[i])
    print(f"Question:{input_question[i]}\nAnswer:{input_answer[i]}\n{sims}\n")

In [211]:
input_queries = ['怎么攻击别人？']
q_embedding = modelB.encode(input_queries)
results = util.semantic_search(query_embeddings = q_embedding,corpus_embeddings= emb_answer,top_k=3)
results

[[{'corpus_id': 93, 'score': 0.4162951409816742},
  {'corpus_id': 90, 'score': 0.33781808614730835},
  {'corpus_id': 76, 'score': 0.29779693484306335}]]

In [212]:
for ret in results[0]:
    print(f"{all_datas[ret['corpus_id']]} score:{ret['score']}")

('全力交锋--战斗阶段开始后，跨服到其它战区后，除了攻打首府和两个2个王者遗迹-守卫外（其他遗迹和发射塔为和平状态，不可攻击），还可以攻打该服玩家的城堡，采集田和强化弱化塔。', '战斗阶段是否可以攻击其他战区指挥官？') score:0.4162951409816742
('玩家拥有建筑的占领权后，拥有【先锋】权限的玩家，可以开启对决。开启对决后，该玩家在此建筑中的所有部队都将会进入对决状态中，其他势力的拥有【先锋】权限的玩家，可以发起对决；当击败玩家所有进入对决状态的部队后，视为攻击成功。对决连续守住一定的攻击次数，或守住规定的时间，则对决成功，否则对决失败。对决获胜，可获得一定时间的无敌时间，并立即摧毁一定的城防值；对决失败，则击败对决的玩家获得建筑的控制权。', '先锋对决怎么玩，胜负规则是什么？') score:0.33781808614730835
('您好，破浪突袭者BW-3的控制技能主要体现在 技能中有一项是潮汐震荡，如果被击目标处在攻击力降低状态，50%概率对其附加[眩晕]（无法普攻和释放主动技能）。从技能定位来看，其更偏向攻击，但防守效果也不错。', '小蓝是拥有控制技能的重装机兵，这个控制技能怎么理解？它更适合攻击还是防守？') score:0.29779693484306335


# 使用pre trained 模型对比

In [136]:
from transformers import AutoTokenizer,AutoModel
import torch

In [137]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [138]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0].to(device) #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float().to(device)
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def load_model(model_location):    
    tokenizer = AutoTokenizer.from_pretrained(model_location)
   
    model = AutoModel.from_pretrained(
        model_location, 
        # device_map="balanced_low_0", 
        #load_in_8bit=True
    )
    # load the model on GPU
    model.to(device) 
    model.eval()
    
    return model, tokenizer

In [171]:
model_location = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
modelA,tokenizer  = load_model(model_location)

In [172]:
encoded_input = tokenizer(input_question, padding=True, truncation=True, max_length=128, return_tensors='pt').to(device)

In [173]:
with torch.no_grad():
    model_output = modelA(**encoded_input)

# Perform pooling. In this case, max pooling.
sentence_embeddings_q = mean_pooling(model_output, encoded_input['attention_mask']).to(device).cpu().numpy()

In [174]:
encoded_input = tokenizer(input_answer, padding=True, truncation=True, max_length=128, return_tensors='pt').to(device)
with torch.no_grad():
    model_output = modelA(**encoded_input)

# Perform pooling. In this case, max pooling.
sentence_embeddings_a = mean_pooling(model_output, encoded_input['attention_mask']).to(device).cpu().numpy()

In [ ]:
for i in range(len(input_question)):
    sims = similarity(sentence_embeddings_a[i],sentence_embeddings_q[i])
    print(f"Question:{input_question[i]}\nAnswer:{input_answer[i]}\n{sims}\n")

In [213]:
input_queries = ['怎么攻击别人?']
encoded_input = tokenizer(input_queries, padding=True, truncation=True, max_length=128, return_tensors='pt').to(device)
with torch.no_grad():
    model_output = modelA(**encoded_input)

q_embedding = mean_pooling(model_output, encoded_input['attention_mask']).to(device).cpu().numpy()

In [214]:
results = util.semantic_search(query_embeddings = q_embedding,corpus_embeddings= sentence_embeddings_a,top_k=3)
results
for ret in results[0]:
    print(f"{all_datas[ret['corpus_id']]} score:{ret['score']}")

('您好，[援护]效果不分担燃烧伤害（除燃烧伤害外，都可以分担）。拥有点燃技能的英雄目前有寂，914，马克西莫，异化娜迪亚等。', '尤里卡突袭者的[援护]效果是否能分担各种英雄技能打出的伤害？') score:0.44886574149131775
('玩家拥有建筑的占领权后，拥有【先锋】权限的玩家，可以开启对决。开启对决后，该玩家在此建筑中的所有部队都将会进入对决状态中，其他势力的拥有【先锋】权限的玩家，可以发起对决；当击败玩家所有进入对决状态的部队后，视为攻击成功。对决连续守住一定的攻击次数，或守住规定的时间，则对决成功，否则对决失败。对决获胜，可获得一定时间的无敌时间，并立即摧毁一定的城防值；对决失败，则击败对决的玩家获得建筑的控制权。', '先锋对决怎么玩，胜负规则是什么？') score:0.4420698285102844
('您好，破浪突袭者BW-3的控制技能主要体现在 技能中有一项是潮汐震荡，如果被击目标处在攻击力降低状态，50%概率对其附加[眩晕]（无法普攻和释放主动技能）。从技能定位来看，其更偏向攻击，但防守效果也不错。', '小蓝是拥有控制技能的重装机兵，这个控制技能怎么理解？它更适合攻击还是防守？') score:0.43532323837280273


In [2]:
input_queries = ['中国的首都在哪里？','中国的首都在北京']
encoded_input = tokenizer(input_queries, padding=True, truncation=True, max_length=128, return_tensors='pt').to(device)
with torch.no_grad():
    model_output = modelA(**encoded_input)

embedding = mean_pooling(model_output, encoded_input['attention_mask']).to(device).cpu().numpy()

NameError: name 'tokenizer' is not defined

# 部署模型到sagemaker

In [12]:
!pip install -U sagemaker

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 851.8/851.8 kB 8.3 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
  Using cached PyYAML-6.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (682 kB)
  Created wheel for sagemaker: filename=sagemaker-2.169.0-py2.py3-none-any.whl size=1158252 sha256=0ec721d00f4427b64ea6d9d4e052bb5989328fde366bb4acd16933fb90d67f82
  Stored in directory: /root/.cache/pip/wheels/42/17/69/c2089332a0db669b4a27888e1d76e825168014112d5eb44231
Successfully built sagemaker
  Attempting uninstall: PyYAML
    Found existing installation:

## 2. 把模型拷贝到S3为后续部署做准备

In [13]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

In [14]:
s3_model_prefix = "LLM-RAG/workshop/finetuned-sentence2emb-model"  # folder where model checkpoint will go
model_snapshot_path = "./finetuned-sentence-embedding"
s3_code_prefix = "LLM-RAG/workshop/finetuned-sentence2emb_deploy_code"
print(f"s3_code_prefix: {s3_code_prefix}")
print(f"model_snapshot_path: {model_snapshot_path}")

s3_code_prefix: LLM-RAG/workshop/finetuned-sentence2emb_deploy_code
model_snapshot_path: ./finetuned-sentence-embedding


In [15]:
!aws s3 cp --recursive {model_snapshot_path} s3://{bucket}/{s3_model_prefix}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
upload: finetuned-sentence-embedding/config_sentence_transformers.json to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/finetuned-sentence2emb-model/config_sentence_transformers.json
upload: finetuned-sentence-embedding/1_Pooling/config.json to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/finetuned-sentence2emb-model/1_Pooling/config.json
upload: finetuned-sentence-embedding/README.md to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/finetuned-sentence2emb-model/README.md
upload: finetuned-sentence-embedding/config.json to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/finetuned-sentence2emb-model/config.json
upload: finetuned-senten

### 3. 模型部署准备（entrypoint脚本，容器镜像，服务配置）

In [16]:
inference_image_uri = (
    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117"
)

#中国区需要替换为下面的image_uri
# inference_image_uri = (
#     f"727897471807.dkr.ecr.{region}.amazonaws.com.cn/djl-inference:0.21.0-deepspeed0.8.3-cu117"
# )

print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 763104351884.dkr.ecr.us-east-2.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117


In [17]:
!mkdir -p sentence2emb_deploy_code

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [18]:
%%writefile sentence2emb_deploy_code/model.py
from djl_python import Input, Output
import torch
import logging
import math
import os
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, AutoModel

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'--device={device}')


def load_model(properties):
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")
    
    tokenizer = AutoTokenizer.from_pretrained(model_location)
   
    model = AutoModel.from_pretrained(
        model_location, 
        # device_map="balanced_low_0", 
        #load_in_8bit=True
    )
    # load the model on GPU
    model.to(device) 
    model.requires_grad_(False)
    model.eval()
    
    return model, tokenizer


model = None
tokenizer = None
generator = None

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0].to(device) #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float().to(device)
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


def handle(inputs: Input):
    global model, tokenizer
    if not model:
        model, tokenizer = load_model(inputs.get_properties())

    if inputs.is_empty():
        return None
    data = inputs.get_as_json()
    
    input_sentences = data["inputs"]
    params = data["parameters"]
    logging.info(f"inputs: {input_sentences}")
    logging.info(f"parameters: {params}")
    
    encoded_input = tokenizer(input_sentences, padding=True, truncation=True, return_tensors='pt').to(device)
    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling. In this case, max pooling.
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask']).to(device).cpu().numpy()

#     # preprocess
#     input_ids = tokenizer(input_sentences, return_tensors="pt").input_ids
#     # pass inputs with all kwargs in data
#     if params is not None:
#         outputs = model.generate(input_ids, **params)
#     else:
#         outputs = model.generate(input_ids)

#     # postprocess the prediction
#     prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    result = {"sentence_embeddings": sentence_embeddings}
    return Output().add_as_json(result)

Writing sentence2emb_deploy_code/model.py


#### Note: option.s3url 需要按照自己的账号进行修改

In [19]:
%%writefile sentence2emb_deploy_code/serving.properties
engine=Python
option.tensor_parallel_degree=1
option.s3url = s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/finetuned-sentence2emb-model/

Writing sentence2emb_deploy_code/serving.properties


In [20]:
!rm s2e_model.tar.gz
!cd sentence2emb_deploy_code && rm -rf ".ipynb_checkpoints"
!tar czvf s2e_model.tar.gz sentence2emb_deploy_code

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
sentence2emb_deploy_code/
se

In [21]:
s3_code_artifact = sess.upload_data("s2e_model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/finetuned-sentence2emb_deploy_code/s2e_model.tar.gz


### 4. 创建模型 & 创建endpoint

In [25]:
from sagemaker.utils import name_from_base
import boto3

model_name = name_from_base("finetuned-paraphrase") #Note: Need to specify model_name
print(model_name)
print(f"Image going to be used is ---- > {inference_image_uri}")

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact
    },
    
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

finetuned-paraphrase-2023-07-04-16-08-10-303
Image going to be used is ---- > 763104351884.dkr.ecr.us-east-2.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117
Created Model: arn:aws:sagemaker:us-east-2:946277762357:model/finetuned-paraphrase-2023-07-04-16-08-10-303


###  如果批量创建索引量较多，建议改成"InstanceType": "ml.g4dn.xlarge",

In [26]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.m5.2xlarge",
            "InitialInstanceCount": 1,
            # "VolumeSizeInGB" : 400,
            # "ModelDataDownloadTimeoutInSeconds": 2400,
            "ContainerStartupHealthCheckTimeoutInSeconds": 10*60,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-2:946277762357:endpoint-config/finetuned-paraphrase-2023-07-04-16-08-10-303-config',
 'ResponseMetadata': {'RequestId': '1c82ff07-2640-43a3-8e23-714ca44099af',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1c82ff07-2640-43a3-8e23-714ca44099af',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '132',
   'date': 'Tue, 04 Jul 2023 16:08:11 GMT'},
  'RetryAttempts': 0}}

In [27]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-east-2:946277762357:endpoint/finetuned-paraphrase-2023-07-04-16-08-10-303-endpoint


In [33]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: InService
Arn: arn:aws:sagemaker:us-east-2:946277762357:endpoint/finetuned-paraphrase-2023-07-04-16-08-10-303-endpoint
Status: InService


## 5. 模型测试

In [34]:
def get_vector_by_sm_endpoint(questions, sm_client, endpoint_name):
    parameters = {
        "max_new_tokens": 50,
        "temperature": 0,
        "min_length": 10,
        "no_repeat_ngram_size": 2,
    }

    response_model = sm_client.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=json.dumps(
            {
                "inputs": questions,
                "parameters": parameters
            }
        ),
        ContentType="application/json",
    )
    json_str = response_model['Body'].read().decode('utf8')
    json_obj = json.loads(json_str)
    embeddings = json_obj['sentence_embeddings']
    return embeddings

In [38]:
prompts1 = """专属技能碎片在哪里获得？"""
prompts1 = """中国首都在哪里？"""

emb1 = get_vector_by_sm_endpoint(prompts1, smr_client, endpoint_name)

In [39]:
prompts2 = """专属技能碎片可以通过多种途径获得，例如礼包商城-特惠礼包界面可以购买专属技能碎片礼包"""
emb2 = get_vector_by_sm_endpoint(prompts2, smr_client, endpoint_name)

In [40]:
util.cos_sim(emb1,emb2)

tensor([[-0.0653]])